In [1]:
#Importing the Libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string

In [2]:
data = pd.read_csv('messages.csv')

In [3]:
data.head()

subject  \
0            job posting - apple-iss research center   
1                                                NaN   
2  query : letter frequencies for text identifica...   
3                                               risk   
4                           request book information   

                                             message  label  
0  content - length : 3386 apple-iss research cen...      0  
1  lang classification grimes , joseph e . and ba...      0  
2  i am posting this inquiry for sergei atamas ( ...      0  
3  a colleague and i are researching the differin...      0  
4  earlier this morning i was on the phone with a...      0

In [4]:
#Checking the shape of data
data.shape

(2893, 3)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2893 entries, 0 to 2892
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   subject  2831 non-null   object
 1   message  2893 non-null   object
 2   label    2893 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 67.9+ KB


In [6]:
#Dropping the duplicates rows
data.drop_duplicates(inplace=True)

In [7]:
#Shows the new shape
data.shape

(2876, 3)

In [8]:
#Checking the null values
data.isnull().sum()

subject    62
message     0
label       0
dtype: int64

In [9]:
#Filling the NaN values in subject columns with 'No Subject'
data["subject"].fillna("No subject", inplace = True)

In [10]:
data.isnull().sum()

subject    0
message    0
label      0
dtype: int64

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sumo2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
data.head()

subject  \
0            job posting - apple-iss research center   
1                                         No subject   
2  query : letter frequencies for text identifica...   
3                                               risk   
4                           request book information   

                                             message  label  
0  content - length : 3386 apple-iss research cen...      0  
1  lang classification grimes , joseph e . and ba...      0  
2  i am posting this inquiry for sergei atamas ( ...      0  
3  a colleague and i are researching the differin...      0  
4  earlier this morning i was on the phone with a...      0

In [22]:
def process_text(message):
    #1-Removing the punctuation
    nopunc = [char for char in message if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    #2-Removing the stopwords
    clean_word = [word for word in nopunc.split() if word.lower not in stopwords.words('english')]
    #3-Return Clean Word
    return clean_word

In [23]:
data['message'].head().apply(process_text)

0    [content, length, 3386, appleiss, research, ce...
1    [lang, classification, grimes, joseph, e, and,...
2    [i, am, posting, this, inquiry, for, sergei, a...
3    [a, colleague, and, i, are, researching, the, ...
4    [earlier, this, morning, i, was, on, the, phon...
Name: message, dtype: object

In [25]:
#Converting a collection of text to a matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer=process_text).fit_transform(data['message'])

In [26]:
#Spliting the dadta into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_bow,data['label'],test_size = 0.2, random_state = 0)

In [27]:
#Get the shape of the message_bow
message_bow.shape

(2876, 64806)

In [28]:
#Create and Train the Naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train,y_train)

In [31]:
#Print the predicted value
print(classifier.predict(X_train))

#Print the actual value
print(y_train.values)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [37]:
#Evaluating the model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train,pred))
print()
print('Confusion Matrix: \n', confusion_matrix(y_train,pred))
print()
print('Accuracy Score: \n', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1914
           1       0.98      0.99      0.99       386

    accuracy                           1.00      2300
   macro avg       0.99      1.00      0.99      2300
weighted avg       1.00      1.00      1.00      2300


Confusion Matrix: 
 [[1905    9]
 [   2  384]]

Accuracy Score: 
 0.9952173913043478
